In [1]:
import pandas as pd
import geopandas as gpd
import folium 
import os
from folium import plugins
from folium.plugins import HeatMap

In [2]:
# Set the parent directory
parentDirectory = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
# Load the mode
os.chdir(parentDirectory)

In [3]:
# Open csv
df = pd.read_csv(parentDirectory+'/data/raw/data.csv')

In [4]:
# Create geodf
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

# Set crs
gdf.crs = 'epsg:4326'

In [5]:
#gdf.head()
df['Month'] = df['Month'].apply('{:0>2}'.format)
df['Day'] = df['Day'].apply('{:0>2}'.format)

In [6]:
df['YearMonthDay'] = df['Year'].astype(str) + df['Month'].astype(str) + df['Day'].astype(str)

In [7]:
df['YearMonthDay'] = df['YearMonthDay'].astype(int)

In [9]:
df_sorted = df.sort_values(['YearMonthDay'])

In [11]:
YearMonthDay = df_sorted['YearMonthDay'].unique()

In [12]:
map_hooray = folium.Map(location=[51.70422, 5.62723],
                        tiles = "Stamen Toner",
                        zoom_start = 13) 


# Ensure you're handing it floats
df_sorted['Latitude'] = df_sorted['Latitude'].astype(float)
df_sorted['Longitude'] = df_sorted['Longitude'].astype(float)

# Create new heat df
heat_df = df_sorted[['Latitude', 'Longitude']]

# Create weight column, using date
heat_df['Weight'] = df_sorted['YearMonthDay']
#heat_df['Weight'] = heat_df['Weight']
heat_df = heat_df.dropna(axis=0, subset=['Latitude','Longitude', 'Weight'])


# List comprehension to make out list of lists
heat_data = [[[row['Latitude'],row['Longitude']] for index, row in heat_df[heat_df['Weight'] == i].iterrows()] for i in YearMonthDay]


# Plot it on the map
hm = plugins.HeatMapWithTime(heat_data, auto_play=True, max_opacity=0.8, radius = 25)
hm.add_to(map_hooray)
# Display the map
map_hooray

<ipython-input-12-90d935d6bee0>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  heat_df['Weight'] = df_sorted['YearMonthDay']


In [13]:
map_hooray.save(parentDirectory+"/data/processed/HeathMapPerDag.html")